In [2]:
from autopycoin.data import random_ts
from autopycoin.models import PoolNBEATS, create_interpretable_nbeats
from autopycoin.dataset import WindowGenerator
import tensorflow as tf
import pandas as pd
import numpy as np

print(
    3
    * (
        1
        + tf.cos(1 * (1 / 3) * 2 * np.pi)
        + tf.cos(2 * (1 / 3) * 2 * np.pi)
        + tf.sin(1 * (1 / 3) * 2 * np.pi)
        + tf.sin(2 * (1 / 3) * 2 * np.pi)
    ).numpy()
)

data = random_ts(n_steps=1000,
                 trend_degree=2,
                 periods=[10],
                 fourier_orders=[10],
                 trend_mean=0,
                 trend_std=1,
                 seasonality_mean=0,
                 seasonality_std=1,
                 batch_size=1,
                 n_variables=1,
                 noise=True,
                 seed=42)
data = pd.DataFrame(data[0].numpy(), columns=['test'])

w = WindowGenerator(
       input_width=70,
       label_width=10,
       shift=10,
       test_size=50,
       valid_size=10,
       flat=True,
       batch_size=32,
       preprocessing=lambda x,y: (x, (x, y))
    )

w = w.from_array(data=data,
       input_columns=['test'],
       label_columns=['test'])

nbeats_model = lambda label_width: create_interpretable_nbeats(label_width=label_width, 
seasonality_n_neurons=20, trend_n_neurons=20)

model = PoolNBEATS(
            label_width=10,
            n_models=1,
            nbeats_models=nbeats_model,
            )
model.compile(tf.keras.optimizers.Adam(
   learning_rate=0.0015, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
   name='Adam'), loss=['mse', 'mae'], metrics=['mae'])
history = model.fit(w.train, validation_data=w.valid, epochs=1, verbose=1)

model.predict(w.test.take(1))

0.0


In [3]:
model.predict(w.test.take(1))[1]

array([[-1.53721452e+00,  1.78958249e+00, -1.21782112e+00,
        -3.20073891e+00,  1.55886352e-01, -1.34092057e+00,
         1.98587322e+00, -1.02152324e+00, -3.00444388e+00,
         3.52180362e-01],
       [ 2.04076385e+00,  2.53998256e+00,  9.28356528e-01,
        -2.02514267e+00,  1.25421739e+00,  1.60430026e+00,
         2.10351682e+00,  4.91894394e-01, -2.46160603e+00,
         8.17755461e-01],
       [ 8.08135331e-01,  4.87694681e-01,  1.45888066e+00,
         1.41493213e+00,  7.28568912e-01,  1.36462307e+00,
         1.04417968e+00,  2.01536417e+00,  1.97142029e+00,
         1.28505325e+00],
       [-4.39251304e-01, -1.80525422e+00,  1.84421921e+00,
         2.52435112e+00, -5.39795101e-01,  1.48077905e-01,
        -1.21792698e+00,  2.43154502e+00,  3.11168051e+00,
         4.75343615e-02],
       [-1.55507922e+00, -1.95048666e+00, -2.80320978e+00,
        -1.66960049e+00,  2.95541644e-01, -1.12819481e+00,
        -1.52360702e+00, -2.37632608e+00, -1.24271560e+00,
         7.

In [ ]:
        # aggregation if possible
        if all(inputs_reconstructed[0].shape == tensor.shape for tensor in inputs_reconstructed[1:]):
            inputs_reconstructed = self._fn_agg(inputs_reconstructed, axis=-1)

        # aggregation if possible
        if all(outputs[0].shape == tensor.shape for tensor in outputs[1:]):
            outputs = self._fn_agg(outputs, axis=-1)